In [2]:
import pypsa
import numpy as np
import pandas as pd
import pickle
from pathlib import Path

from _helpers import configure_logging
import logging
logger = logging.getLogger(__name__)

In [7]:
def create_network():
    """Create the pypsa network scaffolding for the ESC."""
    
    # Modify PyPSA 'Link' component to allow for 2 output busses by overwriting component_attrs
    # c.f. https://www.pypsa.org/examples/chp-fixed-heat-power-ratio.html
    
    # Load additional components
    with open(snakemake.input["additional_components"], "rb") as f:
        override_component_attrs = pickle.load(f)

    # Create network with modified link-component
    network = pypsa.Network(override_component_attrs=override_component_attrs)

    # Load network components from csv files
    network.import_from_csv_folder(snakemake.input["network"])
    
    # Equally weighted snapshots, year defined via config
    year = snakemake.config["scenario"]["year"]
    # Handle leap year by dropping 29th of Februray
    snapshots = pd.date_range(str(year),str(year+1), freq="H", closed="left")
    snapshots = filter(lambda x: not((x.month==2) & (x.day==29)), list(snapshots))
    snapshots = [x.strftime('%Y-%m-%d %H:%M:%S') for x in snapshots]
    network.set_snapshots(snapshots=snapshots)
    
    return network

def attach_efficiencies(network):
    """Attach dedicated efficiencies from file.
    
    The efficiencies are from an additional csv file and added to the links in the pypsa network
    Format for efficiencies.csv file:
    * "from" and "to" must substrings of the bus names
    * "process" must be a substring of the name of the link
    
    Return
    ------
    network : pypsa.network
        network with external efficiencies attached to all links.
    
    """
    efficiencies = pd.read_csv(snakemake.input["efficiencies"])

    df = network.links
    buses = list(filter(lambda x: x.startswith("bus"), df.columns))
    assert len(buses) == 3, "Only a maximum of three buses are supported per link."

    for idx, row in efficiencies.iterrows():

        # short-hand for simplicity
        df = network.links

        # determine relevant links
        df = df[df.index.str.contains(row.process, regex=False)]

        # Check pair-wise combinations in both direction and set correct efficiency
        for bus, efficiency in zip(["bus1", "bus2"],["efficiency", "efficiency2"]):
            df.loc[df["bus0"].str.contains(row["from"], regex=False) &
                   df[bus].str.contains(row["to"], regex=False), efficiency] = row["efficiency"]

            df.loc[df["bus0"].str.contains(row["to"], regex=False) &
                   df[bus].str.contains(row["from"], regex=False), efficiency] = 1./row["efficiency"]

        # Mirror back change to original df
        network.links.update(df)
    
    return network

In [9]:
if __name__ == "__main__":

    configure_logging(snakemake)
    
    Path(snakemake.output["network"]).mkdir(parents=True,exist_ok=True)
    
    network = create_network()
    attach_efficiencies(network)
    

    # Save network as csv because netcdf is not working
    # see https://github.com/PyPSA/PyPSA/issues/204
    network.export_to_csv_folder(snakemake.output["network"])